In [1]:
# dask
import dask
from dask.distributed import Client
from dask.distributed import deploy
from dask_jobqueue import PBSCluster

# standard lib
from pathlib import Path
import io

# image processing
from PIL import Image

# numerical libraries
import numpy

In [2]:
cluster = deploy.local.LocalCluster(n_workers=2)

In [3]:
client = Client(cluster)

In [4]:
@dask.delayed
def load_multiframe_tiff(p):
    im = Image.open(p)
    arr = numpy.empty(shape=(im.n_frames, im.height, im.width), dtype=float)
    for i in range(im.n_frames):
        im.seek(i)
        arr[i] = numpy.array(im)
    return arr

In [6]:
images = []
for p in Path("/home/sanderth/debug_images").glob("*.tiff"):
    images.append(load_multiframe_tiff(str(p)))

In [ ]:
from skimage import img_as_float, img_as_uint, filters, exposure, morphology, segmentation, measure
from skimage.restoration import denoise_nl_means
import numpy as np
from PIL import Image

In [ ]:
def denoised_felzenszwalb_otsu_mask(im):

    data = np.array(im)
    mask = np.empty(data.shape, dtype=bool)
    channels = data.shape[0]

    for i in range(channels):
            img = np.array(data[i])

            # Non-local mean denoising
            if i not in noisy_channels:
                img = denoise_nl_means(np.array(img), multichannel=False, patch_size=nlm_psize,
                                     patch_distance=nlm_pdist, fast_mode=True)


            # Felzenszwalb segmentation
            segment_fz = segmentation.felzenszwalb(img, sigma=fz_sigma, scale=fz_scale)

            # Avoid threshold calculation on low signals
            if len(np.unique(segment_fz)) <= 1:
                mask[i] = segment_fz
                continue

            # Calculation of Otsu threshold
            threshold = filters.threshold_otsu(segment_fz)

            # Convertion to Boolean mask with Otsu threshold
            mask_denoized = segment_fz > threshold

            mask[i] = mask_denoized

    return mask.tolist()

In [7]:
%time
pixels = dask.compute(*images)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [12]:
client.close()
cluster.close()